In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/DeepRL project

Mounted at /content/drive
/content/drive/MyDrive/DeepRL project


In [2]:
from packaging import version

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
import tensorboard as tb

In [3]:
from tensorboard.backend.event_processing import event_accumulator
import pandas as pd


def parse_tensorboard(path, scalars):
    """returns a dictionary of pandas dataframes for each requested scalar"""
    ea = event_accumulator.EventAccumulator(
        path,
        size_guidance={event_accumulator.SCALARS: 0},
    )
    _absorb_print = ea.Reload()
    # make sure the scalars are in the event accumulator tags
    try:
        assert all(
            s in ea.Tags()["scalars"] for s in scalars
        ), "some scalars were not found in the event accumulator"
    except:
        print(path)
        raise Exception()
    return {k: pd.DataFrame(ea.Scalars(k)) for k in scalars}

In [4]:
import os
r = {}
for algo in ["A2C", "DQN", "PPO"]:
    if algo not in r:
        r[algo] = {}
    for env in ["BeamRider","Breakout","Enduro","SpaceInvaders","Pong","Qbert","Seaquest"]:
        if env not in r[algo]:
            r[algo][env] = {'vals':[], 'mean':0, 'sd':0}
        root = f"./{algo}_{env}"
        for dir in os.listdir(root):
            if dir[0]=='.':
                continue
            path = f"{root}/{dir}/"
            v = parse_tensorboard(path, ["rollout/ep_rew_mean"])['rollout/ep_rew_mean'].iat[-1,-1]
            r[algo][env]['vals'].append(float(v))
        assert len(r[algo][env]['vals']) == 3
        mean = sum(r[algo][env]['vals']) / 3
        std = (sum([(v-mean)**2 for v in r[algo][env]['vals']]) / 2) ** 0.5
        if std==0:
          psd = 0
        elif env == "Pong":
          psd = std/(20+mean) * 100
        else:
          psd = std/mean * 100
        mean = round(mean, 1)
        std = round(std, 1)
        psd = round(psd, 1)
        r[algo][env]['mean'] = mean
        r[algo][env]['sd'] = std
        #print(f"{mean}±{std}",end="\t")
        print(f"{mean}",end="\t")
    print("")

797.0	22.5	0.0	410.2	-20.3	558.5	568.5	
4791.2	162.1	532.9	709.0	7.5	2212.2	856.1	
843.5	28.1	30.3	407.6	16.4	2640.6	1045.3	
